In [1]:
import lucene

lucene.initVM()

In [2]:
lucene.VERSION

'9.4.1'

In [3]:
import pandas as pd
import re
from tqdm import tqdm

In [4]:
bad_words = pd.read_csv('/Project/data/bad_word.txt', header=None)
# bad_words = pd.read_csv('data/bad_word.txt', header=None)
bad_words_set = set(bad_words[0].values)

In [5]:
bad_words_list= bad_words.values.flatten()

In [6]:
# jigsaw_dataset = pd.read_csv('data/jigsaw_preprocessed_labels.csv',header=0)
# jigsaw_labels = jigsaw_dataset['label']

In [7]:
news_dataset = pd.read_csv('/Project/data/pos_tags_dataset.csv',header=0)
# news_dataset = pd.read_csv('data/pos_tags_dataset.csv',header=0)
# labels = pd.read_csv('data/comments_model1_outputs.txt',header=None)

In [6]:
news_dataset['words']

0        ['a', 'lot', 'of', 'the', 'clients', 'i', 'wor...
1        ['remember', 'when', 'jim', 'cramer', 'a', 'mo...
2        ['the', 'fdic', 'only', 'insures', 'up', 'to',...
3        ['this', 'is', 'a', 'pretty', 'big', 'deal', '...
4        ['if', 'anyone', 's', 'wondering', 'how', 'a',...
                               ...                        
65689    ['they', 'already', 'have', 'enough', 'local',...
65690    ['indeed', 'it', 'is', 'it', 'still', 'is', 'r...
65691    ['amazon', 'did', 'because', 'it', 'is', 'thei...
65692    ['thanks', 'i', 'did', 'not', 'get', 'past', '...
65693    ['this', 'is', 'what', 'happens', 'they', 'bui...
Name: words, Length: 65694, dtype: object

In [8]:
rows_to_ignore = pd.read_csv('/Project/data/rows_to_ignore.txt',header=None)
# rows_to_ignore = pd.read_csv('data/rows_to_ignore.txt',header=None)

In [9]:
news_dataset = news_dataset.drop(news_dataset.index[rows_to_ignore[0].unique()])

In [10]:
labels = []
pattern = r"\b(" + "|".join(bad_words_list) + r")\b"
regex = re.compile(pattern)

for sentence in tqdm(news_dataset['cleaned_comments']):
    match = regex.findall(sentence.lower())
    labels.append(1) if match else labels.append(0)

100%|██████████| 65616/65616 [00:27<00:00, 2363.94it/s]


In [11]:
news_dataset['label'] = labels

In [12]:
news_dataset.to_csv('/Project/data/pos_tags_labels_dataset.csv',index=False)

In [13]:
bad_sentences = news_dataset[news_dataset['label']==1]
good_sentences = news_dataset[news_dataset['label']==0]

In [16]:
# good_sentences.to_csv('/Project/data/good_sentences.csv')
good_sentences.to_csv('/Project/data/good_sentences.csv')
bad_sentences.to_csv('/Project/data/bad_sentences.csv')

In [17]:
bad_sent = bad_sentences[['cleaned_comments','label', 'pos_tag']]
good_sent = good_sentences[['cleaned_comments','label', 'pos_tag']]

In [18]:
bad_comments = []
good_comments = []
for bad_rows in bad_sent.itertuples(index = True):
    bad_comments.append(bad_rows[1])

for good_rows in good_sent.itertuples(index = True):
    good_comments.append(good_rows[1])


Delete indexdir in data before running this

In [25]:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.document import Document, Field, StringField, TextField
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, DirectoryReader
from org.apache.lucene.search import IndexSearcher,ScoreDoc, TopDocs
from org.apache.lucene.queries.mlt import MoreLikeThisQuery
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import Version
import java
import os

# Indexing
index_dir = '/Project/data/indexdir'
# analyzer = StandardAnalyzer()
analyzer = EnglishAnalyzer()

directory = FSDirectory.open(java.nio.file.Paths.get(index_dir))
config = IndexWriterConfig(analyzer)
writer = IndexWriter(directory, config)

if not os.path.exists(index_dir):
    print('Created dir')
    os.mkdir(index_dir)
    doc1 = Document()
    doc1.add(TextField("content", "The quick brown fox jumped over the lazy dog",  Field.Store.YES))
    doc1.add(TextField("pos_tag", "DT NN VB",  Field.Store.YES))
    writer.addDocument(doc1)
    
else:
    # Insert all good sentences to segments
    for index,good_sentence in good_sentences.iterrows():
        doc = Document()
        doc.add(TextField("content", good_sentence['cleaned_comments'], Field.Store.YES))
        doc.add(TextField("pos_tag", good_sentence['pos_tag'],  Field.Store.YES))
        writer.addDocument(doc)

# writer.commit()
writer.close()


In [26]:
from tqdm import tqdm
# Searching
searcher = IndexSearcher(DirectoryReader.open(directory))
analyzer = EnglishAnalyzer()
final_similar_sentences = []
final_pos_tags = []
# Get MoreLikeThisQuery for the first document
# more_like_this = MoreLikeThisQuery("dude my employer uses svb we are dead in the water right now",["content"],analyzer,'content')
for bad_sentence in tqdm(bad_sentences['cleaned_comments']):
    # print(f' BS: {bad_sentence}')
    similar_sentences=[]
    pos_tags_similar_sentences=[]
    pos_tags_bad_sentences=[]
    more_like_this = MoreLikeThisQuery(bad_sentence,["content"],analyzer,"content")
    hits = searcher.search(more_like_this,10).scoreDocs
    # print results
    # print(f"Found {len(hits)} hits:")
    for hit in hits:
        doc = searcher.doc(hit.doc)
        # print(f"{doc['content']}   score: {hit.score}")
        similar_sentences.append(doc['content'])
        pos_tags_similar_sentences.append(doc['pos_tag'])
    final_similar_sentences.append(similar_sentences)
    final_pos_tags.append(pos_tags_similar_sentences)





100%|██████████| 24190/24190 [00:23<00:00, 1041.98it/s]


In [21]:
bad_sentences_similar = pd.DataFrame({'bad_sentences':bad_sentences['cleaned_comments'],'bad_sentences_pos_tags':bad_sentences['pos_tag'],'similar_sentences':final_similar_sentences,'pos_tags_similar_sentences':final_pos_tags})

In [27]:
bad_sentences_similar.to_csv('/Project/data/bad_sentences_similar_pos_tags.csv',index=False)

In [32]:
final_similar_sentences[400]

['i think there is a real possibility that over the long term this ruins stein s career ',
 'it was based on the real life rampart division in the lapd iirc ',
 'that is an ap standards thing not a bias thing ',
 'this one is different as his is based on selling his company to twitter and having wages as part of the compensation package not a normal employment scenario ',
 'really awful title and the article is not much better yikes ap',
 'you know he was seeing the movie planet of the apes in that story right ',
 'how would the ap know they simply report what the agency says ',
 'ruin their childhood to own the libs',
 'almost all of the cops in we own this city are based named after real people they even feature their real [ mugshots ] c c e a e adfeb jumbo x _ap jpg in the intro ',
 'great violent people with ruined lives ']

In [33]:
bad_sentences['cleaned_comments'].iloc[400]

'based and real these apes ruin a lot of shit for the normal ones '